In [ ]:
import ee
import time

### 1. 함수 정의

In [ ]:
def login_gee(project_id):
    """
    Earth Engine 인증 및 초기화
    """
    try:
        ee.Initialize(project=project_id)
        print(f"Successfully initialized with project: {project_id}")
    except Exception as e:
        print(f"Error initializing Earth Engine: {e}")
        print("Authenticating...")
        ee.Authenticate()
        ee.Initialize(project=project_id)

In [ ]:
def get_region_geometry(country_name, state_name):
    """
    FAO GAUL 데이터셋에서 특정 국가의 주(State) 경계를 가져옵니다.
    """
    states = ee.FeatureCollection('FAO/GAUL/2015/level1') \
        .filter(ee.Filter.eq('ADM0_NAME', country_name)) \
        .filter(ee.Filter.eq('ADM1_NAME', state_name))

    return states.geometry()

In [ ]:
def get_sentinel_collection(start_date, end_date, roi, cloud_cover_max=50):
    return (ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED')
            .filterDate(start_date, end_date)
            .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', cloud_cover_max))
            .filterBounds(roi)
            .select(['B2', 'B3', 'B4', 'B8', 'B11']))

def add_ndvi_band(image):
    ndvi = image.normalizedDifference(['B8', 'B4']).rename('NDVI')
    return image.addBands(ndvi)

In [ ]:
def export_image(image, bands, region, description, folder_name, scale=10, max_pixels=1e13):
    """
    이미지를 Google Drive로 내보냅니다.
    Colab/Web 환경이므로 로컬 경로 체크 로직은 제거하고 Drive 폴더명만 지정합니다.
    """
    export_img = image.select(bands).clip(region)

    task = ee.batch.Export.image.toDrive(
        image=export_img,
        description=description,
        folder=folder_name,  # Google Drive 내의 폴더 이름
        scale=scale,
        region=region,
        maxPixels=max_pixels,
        crs='EPSG:4326' # 필요 시 좌표계 지정
    )
    task.start()
    print(f"Started export task: {description} (Folder: {folder_name})")


In [ ]:
def run_export(country_name, state_name, start_date, end_date, suffix):
    """
    개별 지역에 대한 처리 및 내보내기 실행
    """
    print(f"\nProcessing: {country_name} - {state_name}")

    roi = get_region_geometry(country_name, state_name)

    year = start_date.split('-')[0]
    region_label = state_name.replace(" ", "_")

    # 1. 컬렉션 로드
    collection = get_sentinel_collection(start_date, end_date, roi)

    # 2. NDVI 밴드 추가
    collection_ndvi = collection.map(add_ndvi_band)

    # 3. Quality Mosaic (NDVI 값이 가장 높은 픽셀 선택)
    max_ndvi = collection_ndvi.qualityMosaic('NDVI')

    # 4. [수정됨] 데이터 타입 통일 (모든 밴드를 Float32로 변환)
    # Sentinel-2 원본(UInt16)과 NDVI(Float)가 섞여 있어 오류가 발생하므로 전체를 Float로 변환합니다.
    max_ndvi_float = max_ndvi.toFloat()

    folder_name = f"{year}_{region_label}_{suffix}"
    description = f"{year}_{region_label}_{suffix}_Export"

    # 5. 내보내기 실행
    export_image(
        image=max_ndvi_float,
        bands=['B2', 'B3', 'B4', 'B8', 'B11', 'NDVI'],
        region=roi,
        description=description,
        folder_name=folder_name
    )
    time.sleep(1)

In [ ]:
def get_user_inputs():
    """
    사용자로부터 실행 파라미터를 입력받습니다.
    """
    print("="*50)
    print("Google Earth Engine Sentinel-2 Export Tool")
    print("="*50)

    # Project ID 입력
    project_id = input("Enter Google Cloud Project ID (e.g., ee-your-project-id): ").strip()
    if not project_id:
        print("Project ID is required.")
        return None

    # 로그인
    login_gee(project_id)

    # 날짜 및 Suffix 입력
    print("\n[Date Settings]")
    start_date = input("Enter Start Date (YYYY-MM-DD, e.g., 2025-05-01): ").strip()
    end_date = input("Enter End Date (YYYY-MM-DD, e.g., 2025-10-30): ").strip()
    suffix = input("Enter Suffix for folder/file name (e.g., Oct30): ").strip()

    # 국가 및 주 선택
    print("\n[Region Selection]")
    print("1. Use Preset (Select Country -> Auto-select all predefined states)")
    print("2. Manual Input (Enter Country -> Enter State(s))")
    mode = input("Select Mode (1 or 2): ").strip()

    target_list = []

    if mode == '1':
        print("\nAvailable Presets:")
        keys = list(COUNTRY_STATE_MAP.keys())
        for idx, key in enumerate(keys):
            print(f"{idx + 1}. {key}")

        try:
            country_idx = int(input(f"Select Country (1-{len(keys)}): ")) - 1
            selected_country = keys[country_idx]

            print(f"Selected {selected_country}. Queueing all predefined states...")
            for state in COUNTRY_STATE_MAP[selected_country]:
                target_list.append({'country': selected_country, 'state': state})

        except (ValueError, IndexError):
            print("Invalid selection.")
            return None

    elif mode == '2':
        c_name = input("Enter Country Name (exact match with FAO GAUL, e.g., United States of America): ").strip()

        # [수정된 부분] 쉼표로 구분하여 여러 주 입력 받기
        print("Enter State/Region Names separated by comma.")
        print("Example: Iowa, Ohio, Kansas")
        s_input = input("State(s): ").strip()

        if not s_input:
            print("No state entered.")
            return None

        # 쉼표로 나누고 앞뒤 공백 제거
        input_states = [s.strip() for s in s_input.split(',')]

        for s_name in input_states:
            if s_name: # 빈 문자열이 아닌 경우만 추가
                target_list.append({'country': c_name, 'state': s_name})

        print(f"Queued {len(target_list)} states: {input_states}")

    else:
        print("Invalid mode selected.")
        return None

    return {
        'targets': target_list,
        'start_date': start_date,
        'end_date': end_date,
        'suffix': suffix
    }

### 2. 사용

In [ ]:
# 1. 국가 및 주(State) 프리셋 딕셔너리 정의
COUNTRY_STATE_MAP = {
    'United States of America': [
        'Iowa', 'Illinois', 'Minnesota', 'Nebraska', 'Indiana',
        'Kansas', 'South Dakota', 'Ohio', 'Missouri', 'North Dakota',
        'Wisconsin', 'Arkansas'
    ],
    'Brazil': [
        'Mato Grosso', 'Parana', 'Rio Grande do Sul', 'Goias',
        'Mato Grosso do Sul', 'Bahia', 'Sao Paulo', 'Minas Gerais'
    ],
    'Argentina': [
        'Buenos Aires', 'Cordoba', 'Santa Fe', 'Santiago del Estero',
        'Entre Rios', 'La Pampa'
    ]
}

In [ ]:
# 사용자 입력 받기
params = get_user_inputs()

if not params:
    print("Execution aborted due to invalid inputs.")
    exit()

targets = params['targets']
start_date = params['start_date']
end_date = params['end_date']
suffix = params['suffix']

print(f"\nStarting Batch Processing for {len(targets)} regions...")
print(f"Period: {start_date} ~ {end_date}")

for target in targets:
    run_export(
        country_name=target['country'],
        state_name=target['state'],
        start_date=start_date,
        end_date=end_date,
        suffix=suffix
    )

print("\nAll tasks submitted to Earth Engine.")
print("Check the 'Tasks' tab in the Code Editor or use 'ee.batch.Task.list()' to monitor progress.")


Google Earth Engine Sentinel-2 Export Tool
Enter Google Cloud Project ID (e.g., ee-your-project-id): ee-wodudqkqh123
Successfully initialized with project: ee-wodudqkqh123

[Date Settings]
Enter Start Date (YYYY-MM-DD, e.g., 2025-05-01): 2025-05-01
Enter End Date (YYYY-MM-DD, e.g., 2025-10-30): 2025-10-31
Enter Suffix for folder/file name (e.g., Oct30): 1031

[Region Selection]
1. Use Preset (Select Country -> Auto-select all predefined states)
2. Manual Input (Enter Country -> Enter State(s))
Select Mode (1 or 2): 2
Enter Country Name (exact match with FAO GAUL, e.g., United States of America): United States of America
Enter State/Region Names separated by comma.
Example: Iowa, Ohio, Kansas
State(s): Iowa, Kansas
Queued 2 states: ['Iowa', 'Kansas']

Starting Batch Processing for 2 regions...
Period: 2025-05-01 ~ 2025-10-31

Processing: United States of America - Iowa
Started export task: 2025_Iowa_1031_Export (Folder: 2025_Iowa_1031)

Processing: United States of America - Kansas
Star